To create pickle file

In [3]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 21.6 MB/s 


Source for resources

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


RGB Feature extraction code using I3D

In [ ]:
! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6

Cloning into 'video_features'...
remote: Enumerating objects: 615, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 615 (delta 13), reused 53 (delta 11), pack-reused 555
Receiving objects: 100% (615/615), 163.81 MiB | 24.20 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Checking out files: 100% (71/71), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 11.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.6.3 requires torch<1.12,>=1.7.0, but you have torch 1.2.0+cu92 which is incompatible.
fastai 2.6.3 requires torchvision>=0.8.2,

In [ ]:
%cd video_features

/content/video_features


In [ ]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path, action_on_extraction
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
feature_type = 'i3d'
folder_name='/content/drive/MyDrive/normal'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = [''.join([folder_name, '/', i]) for i in os.listdir(folder_name)]
args.extraction_fps = 32
args.flow_type = 'raft' 
# Load the model
extractor = ExtractI3D(args)
model, class_head = extractor.load_model(device)


  0%|          | 0/1 [14:34<?, ?it/s]


In [ ]:
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    features = extractor.extract(device, model, class_head, video_path)
    [(print(k), print(v.shape), print(v)) for k, v in features.items()]

Extracting for /content/drive/MyDrive/videos/test/Abuse021_x264.mp4


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:878: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool3d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


rgb
(29, 1024)
[[0.17639388 0.16362235 0.08643608 ... 0.20163435 0.09561009 0.36968425]
 [0.15564016 0.14053313 0.09952687 ... 0.28724939 0.11044361 0.07901865]
 [0.14319526 0.12440646 0.07436462 ... 0.20050241 0.0872099  0.18753216]
 ...
 [0.03464679 0.1785682  0.02845623 ... 0.25127223 0.27494383 0.01454003]
 [0.04891974 0.17799912 0.03707507 ... 0.23741707 0.28684804 0.00339542]
 [0.05168566 0.23673862 0.03044268 ... 0.32710525 0.41802523 0.00241226]]
flow
(29, 1024)
[[2.58572884e-02 3.62819880e-02 3.20087783e-02 ... 4.11930569e-02
  8.66518170e-03 1.07087968e-02]
 [5.33869527e-02 3.92315686e-02 4.88899387e-02 ... 1.60588726e-01
  1.52154760e-02 1.76815733e-01]
 [4.57010977e-02 8.37122053e-02 2.82003041e-02 ... 2.38117371e-02
  2.08317372e-03 3.05324644e-02]
 ...
 [3.10656615e-02 2.29972526e-02 2.52324864e-02 ... 2.40503866e-02
  6.74996758e-03 1.67466242e-05]
 [3.49405594e-02 3.20860557e-02 2.99913976e-02 ... 3.62285227e-02
  9.52131860e-03 7.70747196e-03]
 [5.65976948e-02 4.218954

In [ ]:
d={'feature':features['rgb'],'label':1,'num':features['rgb'].shape[0]}

In [ ]:
d

{'feature': array([[0.04317187, 0.32686621, 0.36206684, ..., 0.30663067, 0.20909926,
         0.52908981],
        [0.03676441, 0.38669893, 0.36596587, ..., 0.24567421, 0.21397777,
         0.43830228],
        [0.03689408, 0.39327082, 0.37186712, ..., 0.24284343, 0.28328985,
         0.48415518],
        ...,
        [0.02944066, 0.35002196, 0.36165994, ..., 0.21700065, 0.26356268,
         0.30547273],
        [0.03131689, 0.34578332, 0.36757511, ..., 0.25258383, 0.30103743,
         0.45221433],
        [0.03205033, 0.36976871, 0.32989234, ..., 0.26204252, 0.26245707,
         0.41826856]]), 'label': 1, 'num': 40}

In [ ]:
import pickle5 as pickle
with open('datatest.pkl', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for video_path in args.video_paths:
    print(f'Extracting for {video_path}') 
    features = extractor.extract(device, model, class_head, video_path)
    ls.append({'feature':features['rgb'], 'label':0,'num':features['rgb'].shape[0]})
with open('/content/drive/MyDrive/data/train/data_1.pkl', 'wb') as handle:
  pickle.dump(ls, handle, protocol=pickle.HIGHEST_PROTOCOL)

Changing python torch version

In [2]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1 MB 1.7 kB/s 
     |████████████████████████████████| 8.8 MB 62.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.2.0+cu92 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.2.0+cu92 which 

Model

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import pdb


class Attention(nn.Module):
    def __init__(self, opt):
        super(Attention, self).__init__()
        self.conv = nn.Conv2d(1024, 256, 1)
        self.gcn = GCN(opt, 256, 256)
        self.fc = nn.Linear(256, 1)

    def forward(self, x):
        x = x.view(-1, 1024, 1, 1)
        x = self.conv(x)
        # x = torch.tanh(x)
        x = F.relu(x,inplace=False)
        x = x.clone().view(-1, 256)
        A = self.gcn(x)
        x = self.fc(x)
        mask = torch.sigmoid(x) + 1e-5
        inverse_mask = torch.reciprocal(mask)
        return mask, inverse_mask


class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.fc = nn.Linear(1024, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc(x))


class Network(nn.Module):

    def __init__(self, opt):
        super(Network, self).__init__()
        self.attention = Attention(opt)
        self.classification = Classification()

    def forward(self, x):
        
        mask, inverse_mask = self.attention(x)
        video_feature = torch.sum(x * mask, dim=0, keepdim=True) / torch.sum(mask)
        video_score = self.classification(video_feature)
        inverse_video_feature = torch.sum(x * inverse_mask, dim=0, keepdim=True) / torch.sum(inverse_mask)
        inverse_video_score = self.classification(inverse_video_feature)
        segments_scores = self.classification(x)
        return video_score, inverse_video_score, mask, segments_scores


class GCN(nn.Module):
    def __init__(self, opt, in_channels, out_channels):
        super(GCN, self).__init__()
        self.opt = opt
        if self.opt.C:
            self.theta = nn.Linear(in_channels, in_channels)
            self.phi = nn.Linear(in_channels, in_channels)
        self.conv_d = nn.Linear(in_channels, out_channels)
        if opt.residual:
            self.down = nn.Sequential(nn.Conv1d(in_channels, out_channels, kernel_size=1))

    def forward(self, x):
        t, c = x.size()
        A, M = self.generate_A(t, self.opt.width)
        M = M.detach()
        if self.opt.A:
            A = A.detach()
        else:
            A = 0.
        if self.opt.C:
            theta = self.theta(x)
            phi = self.phi(x)
            C = torch.mm(theta, phi.permute(1, 0))
            if self.opt.CM:
                tmp = torch.exp(C - torch.max(C*M, dim=-1, keepdim=True)[0]) * M
                A += tmp / tmp.sum(dim=-1, keepdim=True)
            else:
                A += F.softmax(C, dim=-1)
        if self.opt.residual:
            out = self.conv_d(torch.bmm(A, x.permute(0, 2, 1)).permute(0, 2, 1)) + self.down(x)
        else:
            out = self.conv_d(torch.mm(A, x))
        return out

    @staticmethod
    def generate_A(dim, width=3):
        A = torch.zeros(dim, dim, device='cuda', requires_grad=False)
        min_value = -(width - 1) // 2
        extent = [min_value+i for i in range(width)]
        for i in range(dim):
            for j in extent:
                if i+j >=0 and i+j <=dim-1:
                  A[i, i+j] = 1.
        M = A
        A = A/A.sum(dim=1, keepdim=True)
        return A, M






**INPUTDATA**
 Module to test and train input is the directory containing pickle file

In [6]:
import os
import random
import numpy as np
import pickle5 as pickle

num_classes = 1


class InputData(object):
    def __init__(self, folder_name, shuffle=True):
        """
        Note: Existing non data files in the folder will raise an exception
        :param folder_name: The name of folder only including data files
        :param shuffle: Whether shuffle data in each files or not
        """
        #print(folder_name)
        self.files_list = [''.join([folder_name, '/', i]) for i in os.listdir(folder_name)]
        
        self.num_file = len(self.files_list)
        self.shuffle = shuffle
        if shuffle:
            self.order_files = random.sample(list(range(self.num_file)), self.num_file)
            self.files_list = [self.files_list[i] for i in self.order_files]
            
        else:
            self.order_files = list(range(self.num_file))
        self.current_file_index = 0
        self.current_video_index = 0
        
        
        with open(self.files_list[0], 'rb') as f:
            self.data = pickle.load(f)
            # print(self.files_list[self.current_file_index])  ##
        self.num_feature = len(self.data)
        if shuffle:
            self.order_feature = random.sample(list(range(self.num_feature)), self.num_feature)
            self.data = [self.data[i] for i in self.order_feature]
        else:
            self.order_feature = list(range(self.num_feature))

    def __check_index(self, size):
        if self.current_video_index + size <= self.num_feature:
            data = self.data[self.current_video_index: self.current_video_index+size]
            self.current_video_index += size
            return data
        else:
            num_excess = self.current_video_index + size - self.num_feature
            data1 = self.data[self.current_video_index: self.num_feature]
            self.current_file_index += 1
            if self.current_file_index == self.num_file:
                if self.shuffle:
                    self.order_files = random.sample(list(range(self.num_file)), self.num_file)
                    self.files_list = [self.files_list[i] for i in self.order_files]
                else:
                    self.order_files = list(range(self.num_file))
                self.current_file_index = 0
            with open(self.files_list[self.current_file_index], 'rb') as f:
                self.data = pickle.load(f)
            self.num_feature = len(self.data)
            if self.shuffle:
                self.order_feature = random.sample(list(range(self.num_feature)), self.num_feature)
                self.data = [self.data[i] for i in self.order_feature]
            else:
                self.order_feature = list(range(self.num_feature))
            data2 = self.data[0: num_excess]
            self.current_video_index = num_excess
            return data1 + data2
    
    def next_batch(self, size):
        data = self.__check_index(size)
        feature = []
        labels = []
        dims = []
        for i in range(size):
            # if data[i]['feature'].shape[0] > 400 and data[i]['feature'].ndim != 1:
            #     feat = data[i]['feature'][0: 400, :]
            #     feature.append(feat)
            # else:
            feature.append(data[i]['feature'])
            
            if data[i]['label'] == 0:
                labels.append([0.])
            else:
                labels.append([1.])
            dims.append(data[i]['num'])
        return feature, np.array(labels, dtype=np.float32), np.array(dims, dtype=np.float32)

TRAIN

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import csv
import argparse

import pdb

train_feature_code_path = '/content/drive/MyDrive/data/train'
test_feature_code_path = '/content/drive/MyDrive/data/test'


def parse_args():
    parser = argparse.ArgumentParser()
    # input for training
    parser.add_argument('--batch_size', default=10, type=int)
    parser.add_argument('--iterations', default=5, type=int)
    parser.add_argument('--epochs', default=100, type=int)
    parser.add_argument('--lr', default=0.5e-3, type=float)
    parser.add_argument('--restore', default=False, type=bool)
    parser.add_argument('--sal_coe', default=0.5, type=float)
    parser.add_argument('--weight_decay', default=0.2e-5, type=float)
    parser.add_argument('--sal_ratio', default=0.3, type=float)
    parser.add_argument('--save_path', default='/content/drive/MyDrive/train_out_3/checkpoints/', type=str)
    parser.add_argument('--gpu_list', default=[0], type=list)
    parser.add_argument('--TEST', default=True, type=bool)
    parser.add_argument('--A', action='store_false')
    parser.add_argument('--C', action='store_true')
    parser.add_argument('--CM', action='store_false')
    parser.add_argument('--residual', action='store_true')
    parser.add_argument('--num_gcn', default=1, type=int)
    parser.add_argument('--width', default=3, type=int)
    args,unknown = parser.parse_known_args()
    return args


def tower_loss(net, features, labels, dims, args):
    loss = []
    inverse_loss = []
    sum_sal_loss = []
    labels = torch.from_numpy(labels).cuda()
    for i in range(len(features)):
        feature = torch.from_numpy(features[i]).cuda()
        feature=feature.type(torch.cuda.FloatTensor) 
        video_score, inverse_video_score, mask, seg_scores = net(feature)
        entropy_loss = F.binary_cross_entropy_with_logits(video_score, labels[i: i+1, :])
        margin = torch.max(torch.tensor(0., device='cuda', requires_grad=False), (torch.sigmoid(seg_scores) - mask) ** 2 - args.sal_ratio ** 2)
        count_nonzero = (margin != 0.).sum().detach().to(torch.float32)
        sal_loss = torch.sum(margin) / (count_nonzero + 1e-6)
        inverse_entropy_loss = labels[i, 0] * F.binary_cross_entropy_with_logits(inverse_video_score, torch.tensor([[0.]], requires_grad=False, device='cuda'))
        loss.append(entropy_loss)
        inverse_loss.append(inverse_entropy_loss + args.sal_coe * sal_loss)
        m=sum(inverse_loss) / args.batch_size
        sum_sal_loss.append(args.sal_coe * sal_loss)
    return sum(loss) / args.batch_size,m , sum(sum_sal_loss) / args.batch_size


def train():
    args = parse_args()
    print('Hyper-parameters:')
    d_args = vars(args)
    for i in d_args:
        print('{}: {}'.format(i, d_args[i]))
    gpu_list = args.gpu_list
    num_gpus = len(gpu_list)
    os.environ["CUDA_VISIBLE_DEVICES"] = ','.join([str(i) for i in gpu_list])
    net = Network(args)
    net.to('cuda')
    net.train()
    optimizer = torch.optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    optimizer_ass = torch.optim.Adam(net.attention.parameters(), lr=args.lr)
    train_data = InputData(train_feature_code_path, shuffle=True)
    if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)
    for i in range(args.epochs):
        print('[*] Current epochs: %d ---' % i)
        sum_loss = 0.
        sum_inverse_loss = 0.0
        sum_sum_sal_loss = 0.0
        for j in range(args.iterations):
            list_features, numpy_labels, numpy_dims = train_data.next_batch(size=args.batch_size)
            loss, inverse_loss, sum_sal_loss = tower_loss(net, list_features, numpy_labels, numpy_dims, args)
            optimizer.zero_grad()
            optimizer_ass.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            torch.autograd.set_detect_anomaly(True)
            inverse_loss.backward()
            optimizer_ass.step()
            sum_loss += loss.item()
            sum_inverse_loss += inverse_loss.item()
            sum_sum_sal_loss += sum_sal_loss.item()
        print('Loss: {:.3f}, Inverse Loss: {:.3f}, sal_loss: {:.3f}'.format(sum_loss / args.iterations, sum_inverse_loss / args.iterations, sum_sum_sal_loss / args.iterations))
        if i > 30:
            print(i)
            torch.save(net.state_dict(), args.save_path + '{}.param'.format(i))
    if args.TEST:
        test(args)


if __name__ == '__main__':
    train()


Hyper-parameters:
batch_size: 10
iterations: 5
epochs: 100
lr: 0.0005
restore: False
sal_coe: 0.5
weight_decay: 2e-06
sal_ratio: 0.3
save_path: /content/drive/MyDrive/train_out_3/checkpoints/
gpu_list: [0]
TEST: True
A: True
C: False
CM: True
residual: False
num_gcn: 1
width: 3
[*] Current epochs: 0 ---
Loss: 0.685, Inverse Loss: 0.575, sal_loss: 0.000
[*] Current epochs: 1 ---
Loss: 0.701, Inverse Loss: 0.527, sal_loss: 0.000
[*] Current epochs: 2 ---
Loss: 0.678, Inverse Loss: 0.554, sal_loss: 0.000
[*] Current epochs: 3 ---
Loss: 0.680, Inverse Loss: 0.527, sal_loss: 0.001
[*] Current epochs: 4 ---
Loss: 0.692, Inverse Loss: 0.483, sal_loss: 0.001
[*] Current epochs: 5 ---
Loss: 0.644, Inverse Loss: 0.587, sal_loss: 0.002
[*] Current epochs: 6 ---
Loss: 0.698, Inverse Loss: 0.439, sal_loss: 0.001
[*] Current epochs: 7 ---
Loss: 0.662, Inverse Loss: 0.524, sal_loss: 0.001
[*] Current epochs: 8 ---
Loss: 0.660, Inverse Loss: 0.523, sal_loss: 0.001
[*] Current epochs: 9 ---
Loss: 0.667

Needed for test

In [8]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 26.6 MB/s 


Test

In [9]:
from sklearn import metrics

import numpy as np
import pickle5 as pickle
import torch
import argparse
import os

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import xlsxwriter
import pdb
from scipy.interpolate import interp1d
from torch.autograd import Variable

def parse_args():
    parser = argparse.ArgumentParser()
    # input for training
    parser.add_argument('--batch_size', default=10, type=int)
    parser.add_argument('--iterations', default=3, type=int)
    parser.add_argument('--epochs', default=80, type=int)
    parser.add_argument('--lr', default=0.5e-3, type=float)
    parser.add_argument('--restore', default=False, type=bool)
    parser.add_argument('--sal_coe', default=0.5, type=float)
    parser.add_argument('--weight_decay', default=0.2e-5, type=float)
    parser.add_argument('--sal_ratio', default=0.3, type=float)
    parser.add_argument('--save_path', default='/content/drive/MyDrive/train_out_3/checkpoints/', type=str)
    parser.add_argument('--gpu_list', default=[0], type=list)
    parser.add_argument('--TEST', default=True, type=bool)

    parser.add_argument('--A', action='store_false')
    parser.add_argument('--B', action='store_false')
    parser.add_argument('--C', action='store_true')
    parser.add_argument('--BM', action='store_false')
    parser.add_argument('--CM', action='store_false')
    parser.add_argument('--residual', action='store_true')
    parser.add_argument('--num_gcn', default=1, type=int)
    parser.add_argument('--width', default=3, type=int)
    args,unknown = parser.parse_known_args()
    return args

def test(args):
    def draw_roc(tpr, fpr, auc):
        plt.figure()
        lw = 2
        plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.4f)' % auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.savefig("./test.png")
        plt.cla()
        plt.clf()
        plt.close()

    gts = np.load('/content/drive/MyDrive/mini/Contrastive-Attention-for-Video-Anomaly-Detection-main/gts.npy',allow_pickle=True)

    test_features = test_data = InputData('/content/drive/MyDrive/data/test', shuffle=False)
    net = Network(args).to('cuda')
    net.eval()
    best_auc = 0
    best_epoch = 0
    with torch.no_grad():
        for i in range(31,args.epochs):
            workbook = xlsxwriter.Workbook('./record.xlsx')
            mask_sheet = workbook.add_worksheet('mask')
            score_sheet = workbook.add_worksheet('score')
            cell_format = workbook.add_format({'font_color': 'red'})
            cell_format2 = workbook.add_format({'font_color': 'blue'})
            print(net.load_state_dict(torch.load('/content/drive/MyDrive/train_out_3/checkpoints/' + '{}.param'.format(i))))
            
            pred = []
            y = []
            for j in range(2): 
                features = torch.from_numpy(test_features.next_batch(1)[0][0]).float().cuda()
                video_scores, inverse_video_scores, masks, segments_scores = net(Variable(features))
                row = np.squeeze(masks.cpu().numpy(), axis=1)
                mask_sheet.write_row(j, 1, row.tolist())
                mask_sheet.write(j, 0, np.mean(row), cell_format2)
                mask_sheet.conditional_format(j, np.argmax(row)+1, j, np.argmax(row)+1, {'type': 'no_errors', 'format': cell_format})
                row = np.squeeze(segments_scores.cpu().numpy(), axis=1)
                score_sheet.write_row(j, 0, row.tolist())
                score_sheet.conditional_format(j, np.argmax(row), j, np.argmax(row), {'type': 'no_errors', 'format': cell_format})
                scores = np.squeeze(segments_scores.cpu().numpy())
                video_score = video_scores.cpu().numpy()
                if video_score[0, 0] < -2:
                    scores += video_score[0, 0]
                x = np.arange(0, scores.shape[0])
                f = interp1d(x, scores, kind='linear', axis=0, fill_value='extrapolate')
                scale_x = np.arange(0, scores.shape[0], 1 / 60)
                pred += list(f(scale_x))
                y += b2[j].tolist()
            fpr, tpr, thresholds = metrics.roc_curve(y, pred)
            thresholds=list(map(lambda x:x-0.002,thresholds))
            auc = metrics.auc(fpr, tpr)
            draw_roc(tpr, fpr, auc)
            print('Epoch: {}, AUC: {}'.format(i, auc))
            if auc > best_auc:
                best_auc = auc
                best_epoch = i
            workbook.close()
    print('Best_Epoch: {}, Best_AUC: {}'.format(best_epoch, best_auc))
    return best_auc


args = parse_args()
print('Hyper-parameters:')
d_args = vars(args)
for i in d_args:
  print('{}: {}'.format(i, d_args[i]))
test(args)

Manually creating groundtruth values

In [1]:
import numpy as np
a=[0 for i in range(22)]
a[2]=1
a[3]=1
a1=[0 for i in range(24)]
a1[21]=a1[22]=a1[23]=1
ra1=[]
ra2=[]
for i in range(len(a)):
  for j in range(60):
    ra1.append(a[i])
for i in range(len(a1)):
  for j in range(60):
    ra2.append(a1[i])
b=np.array(ra1)
b1=np.array(ra2)
b2=[]
b2.append(b)
b2.append(b1)
print(b2)


[array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 1, 1, 1])]
